In [1]:
import h5py
import numpy as np

class BaseReaderElectrophysiology:
    def __init__(self, file_name, channels=None):
        self.file_name = file_name
        self.fp = h5py.File(file_name, 'r',  driver='family',memb_size=2147483648)
        sz = self.fp['data'].shape
        self.nb_channels = sz[0]
        self.nb_samples = sz[1]
        self.ch_names = self.fp.attrs['channelNames'].decode('UTF-8').strip().split(',')
        self.Fs = self.fp.attrs['Fs']
        self.t0 = self.fp.attrs.get('t0', 0)
        self.scale = self.fp.attrs['scale']
        #self.eh = self.fp['data'][0][:]
        #self.ev = self.fp['data'][1][:]
        #self.ps = self.fp['data'][3][:]

        

    def match_channels(self, channel):
       
        if not (isinstance(channel, str)):
            raise ValueError('channel name must be a string')
        
        if not (channel in self.ch_names):
            raise KeyError('channel name not valid. Channel name can only be one of Eye Horizontal, Eye Vertical, Photodiode, PupilSizeIn')
    

        ch_index = self.ch_names.index(channel)
        ch_data = self.fp['data'][ch_index][:]
        print('i did my job')
        
        return ch_data
    
    def get_sample_index(self, t):
        # Returns the sample indices 'idx' for the vector of timestamps 't'.

        #print("the value of t0 is ", self.t0)
        if not (self.t0 > 0):
            raise ValueError('t0 has not been updated in this file!')
        
        idx = np.round(1e-3 * (t - self.t0) * self.Fs) + 1
        idx[idx < 1] = np.nan
        idx[idx > self.nb_samples] = np.nan
        return idx

In [ ]:
import datajoint as dj
import os
import h5py
import scipy.io
import numpy as np
from scipy.signal import decimate
import pandas as pd
from matplotlib import pyplot as plt

In [ ]:
import datajoint as dj
dj.config['database.host'] = 'at-database3.stanford.edu'
dj.config['database.user'] = 'konsti'
dj.config['database.password'] = 'black-coffee-fast-bike'
csrf = dj.create_virtual_module("george_csrf","george_csrf")
#events = (csrf.EyeTracePreprocessed.EyeEventsFullSession() & session_dict).fetch()
acq = dj.create_virtual_module("acq","acq")
ephys = dj.create_virtual_module("ephys","ephys")
stim = dj.create_virtual_module("stimulation", "stimulation")

In [ ]:
key = dict(session_start_time=3638885582960)

In [ ]:
acq.Stimulation() & key

In [ ]:
filename_behavior = '/mnt/mscratch1/Dennis/2019-04-23_12-33-02/2019-04-23_12-33-18/BehaviorData%d.h5'

In [ ]:
br = BaseReaderElectrophysiology(filename_behavior, )

In [ ]:
stim_file = '/mnt/stor01/stimulation/Dennis/2019-04-23_12-33-19/CenterSurroundExperimentSynched.mat'

In [ ]:
stim = scipy.io.loadmat(stim_file,struct_as_record=False, squeeze_me=True)
stimParams = stim['stim']

In [ ]:
monitor_width = stimParams.params.constants.monitorCenter[0]*2
monitor_distance = stimParams.params.constants.monitorDistance
monitor_size = stimParams.params.constants.monitorSize[0]
ep = stimParams.params.trials[0].eyeParams

# Calculate pixels per degree
pixel_per_deg = 1 / (np.degrees(np.arctan(monitor_size / (2 * monitor_distance))) * 2 / monitor_width)

print("read experiment config parameters")

In [ ]:
pixel_per_deg

In [ ]:
eh = np.array(br.match_channels('Eye Horizontal').tolist())
ev = np.array(br.match_channels('Eye Vertical').tolist())

In [ ]:
# downsampling factor is 10: eye link runs at 200hz, and is upsampled to 2khz. so it's downsamples by  factor of 10 again
ds = 10

In [ ]:
eye_horizontal = ((ep[0] + ep[1] * eh * br.scale[1])/pixel_per_deg)[::ds]
eye_vertical = ((ep[0] + ep[1] * ev * br.scale[1])/pixel_per_deg)[::ds]

In [ ]:
plt.plot(eye_horizontal[:1000])
plt.plot(eye_vertical[:1000])

In [ ]:
stim_file = "{}/{}Synched.mat".format(stim_filepath,experiment)